In [ ]:
import pandas as pd
import numpy as np

from konlpy.tag import Okt; okt = Okt()

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

from gensim.models.word2vec import Word2Vec

import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('신문기사모음(9.17)(utf8).csv')
text = data.body

In [ ]:
for i, document in enumerate(text):

    # okt = Okt()
    clean_words = []
    for word in okt.pos(document, stem=True): #어간 추출
        if word[1] not in ['Josa', 'Eomi', 'Punctuation', 'Foreign', 'Suffix' ,'URL', 'Conjunction']: #조사, 어미, 구두점, 외국어/기호, url 제외 
            clean_words.append(word[0])
    # print(clean_words) 
    document = ' '.join(clean_words)
    # print(document) 
    text[i] = document

print(text[0], text[1])

In [ ]:
# tokenize 
okt=Okt()
tokens = []
stopwords = ['하다', '되다', '있다', '들다', '들', '기자', '오다', '돼다', '년']
# stopwords = ['기자', '년']


'''각 기사별로 토큰들 나눠서 리스트로 만들기'''
# [ [1번 기사의 토큰들], [2번 기사의 토큰들], [3번 기사의 토큰들]... [n번 기사의 토큰들] ]
for line in text:
    tokens.append(okt.morphs(line))

# noun -> 각 기사별 명사 토큰 list를 모아놓은 list   (list를 모은 list)


# 새로운 dict tokens_processed 생성 
# tokens_processed = {}
whole = []


# 기사 하나하나 마다 [토큰 리스트]에 담긴 한글자짜리 토큰 제거
temp_index = 0
for lst in tokens:
    ''' 여기서 각 기사 한줄 한줄씩 처리함 '''    
    

    # 딕셔너리에 담을 토큰들 모아놓는 리스트....
    temp_words_list = []
    

    for wd in lst:
        ''' 여기서 기사별 [토큰list] 속 단어(토큰)들을 처리함 '''
        
        if len(wd) < 2:
            # 한글자 짜리면 무시하고 지나침
            continue
            
        if wd in stopwords:
            # 불용어에 해당하는 단어면 무시하고 지나침
            continue
        
        # 둘다 해당하지 않으면 임시 리스트에 담기
        temp_words_list.append(wd)
    
    
    # 필터링한 데이터 저장
    # tokens_processed[temp_index] = temp_words_list
    whole.append(temp_words_list)
    temp_index +=1

    ''' 
      0 : [ 기사1 토큰리스트 ]
      1 : [ 기사2 토큰리스트 ]
      2 : [ 기사3 토큰리스트 ]
    ...
    122 : [ 기사122 토큰리스트 ]
    
    (temp_index)    :  (temp_wors_list) 
    이런 형식으로 dictionary(tokens_processed)에 저장
    
    '''
    
# tokens_processed
# whole[:2]

## 동시출현 기반 연관어 분석
#### 같은 문맥에서 중복되는 단어들은 하나로 처리

In [ ]:
count = {}   #동시출현 빈도가 저장될 dict
for line in whole:
    words = list(set(line))   #단어별로 분리한 것을 set에 넣어 중복 제거하고, 다시 list로 변경
    for i, a in enumerate(words):
        for b in words[i+1:]:
            if a == b: continue   #같은 단어의 경우는 세지 않음
            elif a > b:   #A, B와 B, A가 다르게 세어지는것을 막기 위해 항상 a < b로 순서 고정
                count[b, a] = count.get((b, a), 0) + 1   #키가 이미 딕셔너리에 있는지 확인하고 키가 없으면 기본값으로 설정
            else:
                count[a, b] = count.get((a, b), 0) + 1   

In [ ]:
# 데이터프레임으로 예쁘게 만들기
df = pd.DataFrame.from_dict(count, orient='index')

# [term1, term2, freq] 리스트 생성
list1 = []
for i in range(len(df)):
    list1.append([df.index[i][0], df.index[i][1], df[0][i]])

df2 = pd.DataFrame(list1, columns=['term1', 'term2', 'freq'])   #헤더 만들어서 데이터프레임 생성
df3 = df2.sort_values(by=['freq'], ascending=False)   #frequency 큰 순으로 정렬
df3 = df3.reset_index(drop=True) #index 다시 만들기

df3.head(10)

## 통계적 가중치 기반 연관어 분석
#### 유사도 구하기 위해 단어마다 가중치 할당
- 출현 빈도, tf-idf, tf-icf, cosine similarity, jaccard similarity, overlap imilarity

In [ ]:
vec = TfidfVectorizer()
vector_lines = vec.fit_transform(text) # tfidf 가중치 할당
A = vector_lines.toarray()
A.shape

In [ ]:
A = A.transpose() # 문서-단어 매트릭스를 단어-문서 매트릭스로 전치 (문서 간 유사도 아닌 단어 간 유사도 구하기 위해)
A.shape

In [ ]:
A_sparse = sparse.csr_matrix(A)
similarities_sparse = cosine_similarity(A_sparse, dense_output=False) # cosine similarity 계산
list(similarities_sparse.todok().items())[35000:35010]

In [ ]:
vec.get_feature_names()[1025:1030] # 각 단어 명칭에 접근

In [ ]:
# 데이터프레임으로 예쁘게 만들기
df = pd.DataFrame(list(similarities_sparse.todok().items()), columns=["words", "weight"])

df2 = df.sort_values(by=['weight'], ascending=False) # 단어 페어 간 유사도 높은 순으로 정렬
df2 = df2.reset_index(drop=True)

df3 = df2.loc[np.round(df2['weight']) < 1] # 같은 단어끼리 페어는 유사도 1, 1 미만 페어만 추출
df3 = df3.reset_index(drop=True)
df3.head(10)

## word2vec 기반 연관어 분석
#### 2013년 구글에서 제안, 두 가지 가정 기반
1. 단어의 의미는 그 단어 주변 분포로 이해될 수 있다.
2. 단어의 의미는 단어 벡터 안에 인코딩 될 수 있다.

#### 앞 두개와 다른 점
1. 같은 문맥에서의 1) 출현 빈도 뿐 아니라 2) 단어 위치, 3) 순서에 따라서도 가중치를 다르게 준다.
2. 가중치 산출 후 앞에서와 같이 유사도 공식 이용해 단어간 유사도 산출한다.

#### 두 개 알고리즘
1. CBOW: 주변 단어로 중심단어 예측
2. Skip-gram: 중심 단어로 주변 단어 예측 (window size 따라 반복학습 더 많이 함 -> 더 정확한 경우 많다)

#### 참조
Distributed Representations of Words and Phrases and their Compositionality
(https://arxiv.org/pdf/1310.4546.pdf)

In [ ]:
txt = []
for line in whole:
    txt.extend(list(set(line)))

model = Word2Vec(txt, vector_size=100, window = 5, min_count=2, workers=4, sg=1) # skip-gram, window=주변단어, min_count=전체문서에서 최소출현횟수, workers=cpu 코어수
# model.wv.similarity('게임', '규제')
# model.wv.most_similar(positive=["게임"], topn=5)
model.wv.index_to_key

# Visualization

## Gephi 이용

- Gephi 다운로드 (https://gephi.org/)
-> Java 1.8 이상 필요 (https://github.com/ojdkbuild/ojdkbuild)

- 또는 yEd graph editor 다운로드

In [ ]:
len((np.where(df3_pos['freq']>=25))[0])

In [ ]:
# Centrality

G=nx.Graph()
for i in range(1071):
    #print(pair)
    
    G.add_edge(df3_pos['term1'][i], df3_pos['term2'][i], weight=int(df3_pos['freq'][i]))
# Compute centralities for nodes.
# The degree centrality values are normalized by dividing by the maximum possible degree in a simple graph n-1 where n is the number of nodes in G.
dgr = nx.degree_centrality(G)
btw = nx.betweenness_centrality(G)
cls = nx.closeness_centrality(G)
# itemgetter(0): key 또는 itemgetter(1): value로 sort key, reverse=True (descending order)
sorted_dgr = sorted(dgr.items(), key=operator.itemgetter(1), reverse=True)
sorted_btw = sorted(btw.items(), key=operator.itemgetter(1), reverse=True)
sorted_cls = sorted(cls.items(), key=operator.itemgetter(1), reverse=True)
print("** degree **")
for x in range(20):
    print(sorted_dgr[x])
print("** betweenness **")
for x in range(20):
    print(sorted_btw[x])
print("** closeness **")
for x in range(20):
    print(sorted_cls[x])

In [ ]:
#단어끼리 서로 빈도를 세는 데이터셋을 만들었을 때 Gaphi로 시각화하는 것 전단계: graphml 확장자 형식으로 만들기
class MakeGraphml:
    def make_graphml(self, pair_file, graphml_file):
        out = open(graphml_file, 'w', encoding = 'utf-8')
        entity = []
        e_dict = {}
        count = []
        for i in range(len(pair_file)):
            e1 = pair_file.iloc[i,0]
            e2 = pair_file.iloc[i,1]
            #frq = ((word_dict[e1], word_dict[e2]),  pair.split('\t')[2])
            frq = ((e1, e2), pair_file.iloc[i,2])
            if frq not in count: count.append(frq)   # ((a, b), frq)
            if e1 not in entity: entity.append(e1)
            if e2 not in entity: entity.append(e2)
        print('# terms: %s'% len(entity))
        #create e_dict {entity: id} from entity
        for i, w in enumerate(entity):
            e_dict[w] = i + 1 # {word: id}
        out.write(
            "<?xml version=\"1.0\" encoding=\"UTF-8\"?><graphml xmlns=\"http://graphml.graphdrawing.org/xmlns\" xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xsi:schemaLocation=\"http://graphml.graphdrawing.org/xmlnshttp://graphml.graphdrawing.org/xmlns/1.0/graphml.xsd\">" +
            "<key id=\"d1\" for=\"edge\" attr.name=\"weight\" attr.type=\"double\"/>" +
            "<key id=\"d0\" for=\"node\" attr.name=\"label\" attr.type=\"string\"/>" +
            "<graph id=\"Entity\" edgedefault=\"undirected\">" + "\n")
        # nodes
        for i in entity:
            out.write("<node id=\"" + str(e_dict[i]) +"\">" + "\n")
            out.write("<data key=\"d0\">" + i + "</data>" + "\n")
            out.write("</node>")
        # edges
        for y in range(len(count)):
            out.write("<edge source=\"" + str(e_dict[count[y][0][0]]) + "\" target=\"" + str(e_dict[count[y][0][1]]) + "\">" + "\n")
            out.write("<data key=\"d1\">" + str(count[y][1]) + "</data>" + "\n")
            #out.write("<edge source=\"" + str(count[y][0][0]) + "\" target=\"" + str(count[y][0][1]) +"\">"+"\n")
            #out.write("<data key=\"d1\">" + str(count[y][1]) +"</data>"+"\n")
            out.write("</edge>")
        out.write("</graph> </graphml>") 
        print('now you can see %s' % graphml_file)
        #pairs.close()
        out.close()
gm = MakeGraphml()
graphml_file = '파일명.graphml'
#iloc는 인덱스 index of location 열에서 : 써야 함 (열 전체 보여주려면)
gm.make_graphml(df3.iloc[0:1027,:], graphml_file)

In [ ]:
# # Visualization

# class MakeGraphml:
#     def make_graphml(self, pair_file, graphml_file):
#         out = open(graphml_file, 'w', encoding = 'utf-8')

#         entity = []
#         e_dict = {}
#         count = []
#         for i in range(len(pair_file)):
#             e1 = pair_file.iloc[i, 0]
#             e2 = pair_file.iloc[i, 1]
#             # frq = ((word_dict[e1], word_dict[e2]), pair,split('\t')[2])
#             frq = ((e1, e2), pair_file.iloc[i, 2])
#             if frq not in count: count.append(frq)   # ((a, b), frq)
#             if e1 not in entity: entity.append(e1)
#             if e2 not in entity: entity.append(e2)
#         print('# terms: %s'% len(entity))

#         for i, w in enumerate(entity):
#             e_dict[w] = i + 1   # {word: id}

#         out.write(
#             "<?xml version=\"1.0\" encoding=\"UTF-8\"?><graphml xmlns=\"http://graphml.graphdrawing.org/xmlns\" xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xsi:schemaLocation=\"http://graphml.graphdrawing.org/xmlns/1.0/graphml.xsd\">" 
#             +
#             "<key id=\"d1\" for=\"edge\" attr.name=\"weight\" attr.type=\"double\"/>"
#             +
#             "<key id=\"d0\" for=\"node\" attr.name=\"label\" attr.type=\"string\"/>"
#             +
#             "graph id=\"Entity\" edgedefault=\"undirected\">" + "\n")

#         # nodes
#         for i in entity:
#             out.write("<node id=\"" + str(e_dict[i]) + "\">" + "\n")
#             out.write("<data key=\"d0\">" + i + "</data>" + "\n")
#             out.write("</node>")

#         # edges
#         for y in range(len(count)):
#             out.write("<edge source=\"" + str(e_dict[count[y][0][0]]) + "\" target=\"" + str(e_dict[count[y][0][1]]) + "\">" + "\n")
#             out.write("<data key=\"d1\">" + str(count[y][1]) + "</data>" + "\n")
#             out.write("</edge>")

#         out.write("</graph> </graphml>")
#         print('now you can see %s' % graphml_file)

#         out.close

In [ ]:
# gm = MakeGraphml()

# # 파일경로와 파일명 입력 (여기서는 파일명만)
# graphml_file = '연관어네트워크(10).graphml'

# # 연관도(동시출현 빈도)가 10 초과인 행들만 선정
# gm.make_graphml(df3_pos.iloc[0: (len(np.where(df3_pos['freq'] > 10)[0])), :], graphml_file)

## NetworkX 패키지 이용

중심성이란 그래프 이론에서 쓰이는 용어입니다. 하지만 단어 간의 연관도를 링크로 표현하면 하나의 그래프가 형성되기 때문에 연관성 분석에서 다루겠습니다. 앞에서 단어 페어 간의 연관도를 계산하는 방법들을 배웠습니다. 하지만 전체 단어군에서 개별 단어(노드)의 상대적 중요성을 알 수는 없습니다.

예를 들어 A와 B라는 단어 간의 연관도를 구했지만, 전체 단어 중에서 A라는 단어가 상대적으로 지니는 중요성은 알지 못합니다. 또한, 그 중요성을 어떠한 잣대로 살펴봐야 할지도 여러 가지가 있습니다. 하지만 각 단어별 중심성 계수를 구하면 단어별 상대적 중요성을 중심성 계수 성격별로 구할 수 있습니다. 그 후 이 결과를 이용해 리포트할 수도 있고 시각화 할 때 노드의 크기, 색상, 진하기를 달리할 수도 있습니다. 대표적인 중심성 계수로는 연결 중심성(degree centrality), 매개 중심성(betweenness centrality), 고유벡터 중심성(eigenvector centrality) 등이 있습니다.

- 연결 중심성: 하나의 노드가 직접적으로 몇 개의 노드와 연결돼 있는지 측정합니다. 즉, 하나의 단어와 직접적으로 연관도 링크를 지닌 단어의 개수를 측정합니다. 단순히 1개의 링크 거리만을 고려함으로 국지적인 범위에서만의 역할을 보게 됩니다.

- 근접 중심성: 단순히 1개의 링크 거리만을 고려하는 것은 전체 네트워크상에서의 노드의 영향력을 파악하기 어렵습니다. 따라서 직접적인 연결(1개 링크 거리)뿐만 아니라 간접적인 연결(2개 이상의 링크 거리)까지 포함해서 중심성을 측정합니다. 즉, 특정 단어와 연속적인 링크로 연결되는 모든 단어와의 거리에 따른 평균적인 연관도를 측정하여 글로벌적인 중요성 판단이 가능해집니다.

- 매개 중심성: 해당 노드가 중계자 또는 브로커 역할을 얼마나 잘하는지 측정합니다. 즉, 노드 간 링크를 타고 건너갈 때 핵심적으로 통과해야만 하는 노드를 찾을 때 용이합니다. 매개 중심성이 크다면 네트워크 내의 의사소통의 흐름에 영향을 줄 소지가 많습니다. 매개 중심성은 작업자 간 워크플로우를 분석할 때 매우 용이하지만 텍스트 마이닝에서는 자주 활용되지는 않습니다.

- 고유벡터 중심성: 각 노드마다 중요성을 부과할 때 해당 노드와 연결된 노드들의 중심성을 고려합니다. 즉, 높은 고유벡터 중심성을 가진 노드는 높은 점수를 가진 많은 노드와 연결되었음을 의미합니다. 고유벡터 중심성을 변형한 알고리즘 중 구글 검색 알고리즘 PageRank가 유명합니다.


In [ ]:
# 중심성 척도 계산 위한 그래프 생성
G_pos = nx.Graph()

In [ ]:
# 빈도수 10 이상인 단어쌍에 대해서만 edge(간선) 표현
for i in range((len(np.where(df3_pos['freq'] > 10)[0]))):
    G_pos.add_edge(df3_pos['term1'][i], df3_pos['term2'][i], weigh=int(df3_pos['freq'][i]))

In [ ]:
dgr = nx.degree_centrality(G_pos)   # 연결 중심성
btw = nx.betweenness_centrality(G_pos)   # 매개 중심성
cls = nx.closeness_centrality(G_pos)   # 근접 중심성
egv = nx.eigenvector_centrality(G_pos)   # 고유벡터 중심성
pgr = nx.pagerank(G_pos)   # 페이지랭크

In [ ]:
# 중심성 큰 순서로 정렬
sorted_dgr = sorted(dgr.items(), key = operator.itemgetter(1), reverse = True)
sorted_btw = sorted(btw.items(), key = operator.itemgetter(1), reverse = True)
sorted_cls = sorted(cls.items(), key = operator.itemgetter(1), reverse = True)
sorted_egv = sorted(egv.items(), key = operator.itemgetter(1), reverse = True)
sorted_pgr = sorted(pgr.items(), key = operator.itemgetter(1), reverse = True)

In [ ]:
# 단어 네트워크 그릴 그래프 생성
G = nx.Graph()

In [ ]:
# 페이지 랭크에 따라 두 노드 사이 연관성 결정 (단어쌍의 연관성)
# 연결 중심성으로 계산한 척도에 따라 노드 크기 결정 (단어의 등장 빈도수)

for i in range(len(sorted_pgr)):
    G.add_node(sorted_pgr[i][0], nodesize = sorted_dgr[i][1])

In [ ]:
for i in range((len(np.where(df3_pos['freq'] > 10)[0]))):
    G.add_weighted_edges_from([(df3_pos['term1'][i], df3_pos['term2'][i], int(df3_pos['freq'][i]))])

In [ ]:
# 노드 크기 조정
sizes = [G.nodes[node]['nodesize'] * 5000 for node in G]

In [ ]:
options = {
    'edge_color': '#a2a1b3',
    'width': 1,
    'with_labels': True,
    'font_weight': 'regular',
}

In [ ]:
import matplotlib.font_manager as fm
from matplotlib import rc
font_name = fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

In [ ]:
plt.figure(figsize=(10, 10))

# font_family = 'KoPubDotum' -> 위 셀 지우고 이것만 추가해도 돌아감
nx.draw(G, font_family = font_name, font_size = 16,
        node_size = sizes, node_color = list(pgr.values()), cmap=plt.cm.Pastel1,   # pgr weight에 따라 노드 색상 다르게
        pos = nx.spring_layout(G, k = 3.5, iterations = 100), **options)
        
ax = plt.gca()
# ax.collections[0].set_edgecolor("#0f03ff")   # 노드 테두리 색
plt.show()